<a href="https://colab.research.google.com/github/RitikaDharamkarJ/test_repository/blob/main/Tweets_Predictions_modify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Tweets Prediction**

Stopword removal is a common preprocessing step in NLP tasks such as text classification. Stopwords are words that are very common and do not carry much meaning, such as "the", "and", "a", etc. Removing stopwords helps to reduce the size of the dataset and to focus the analysis on the more meaningful words. The choice of stopwords to remove can depend on the specific task and the dataset being used, and different NLP libraries provide different lists of stopwords that can be used.

!pip install keras

In [1]:
!pip install keras-preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.3 MB/s eta 0:00:00


In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 3.2 MB/s eta 0:00:00


In [3]:
# Importing the required libraries
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.layers import Dense, Input, Embedding, Conv1D, MaxPool1D, GlobalMaxPool1D
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,precision_score,recall_score
from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,precision_score,recall_score

# load your tweet data
tweets_Data = pd.read_csv("/content/Cyberbullying_tweets_cleaned.xlsx")
tweets_Data.head()

In [7]:
#
tweets_Data = pd.read_csv("/content/Cyberbullying_tweets_cleaned.xlsx")
tweets_Data.head()

UnicodeDecodeError: ignored

In [ ]:
tweets_Data.shape

(47692, 2)

In [ ]:
#Let check the classes of the target column
np.unique(tweets_Data["cyberbullying_type"])

array(['age', 'ethnicity', 'gender', 'not_cyberbullying',
       'other_cyberbullying', 'religion'], dtype=object)

#let drop all rows where tageet column = 'age', 'ethnicity', 'gender', 'religion'

#Remove all rows where the value in the column equals 'come'
tweets_Data = tweets_Data[~tweets_Data['cyberbullying_type'].isin(['age', 'ethnicity', 'gender', 'religion'])]

#Verify that the rows have been removed
np.unique(tweets_Data["cyberbullying_type"])

In [ ]:
tweets_Data = tweets_Data.dropna()

In [ ]:
text = tweets_Data.drop("cyberbullying_type", axis=1)
labels = tweets_Data["cyberbullying_type"]


**Calculate metrics**

In [ ]:
def Print_metric(modelname,y_test,y_pred):
    acc = accuracy_score(y_test,y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')
    pre = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test,y_pred,average='macro')

    # Print results
    print("Evaluation metrics of the ", modelname)
    print("Accuracy: ", acc)
    print("F1 Score: ", f1)
    print("Precision: ", pre)
    print("Recall: ", recall)
    print("Confusion Matrix: \n", confusion_matrix(y_test,y_pred))

##**MLP (Multilayer Perceptron)**

Basic implementation of an MLP (Multilayer Perceptron) for predicting cyberbullying from tweets:

In [ ]:
# Preprocess the text data by converting it to lowercase and encoding the labels
text = tweets_Data["tweet_text"].str.lower()
le = LabelEncoder()
labels = le.fit_transform(tweets_Data["cyberbullying_type"])
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(text, labels, test_size=0.2)

In [ ]:
# Convert the text data into numerical representations using a bag-of-words model
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
# Train the MLP classifier
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=0)
mlp.fit(X_train, y_train)

MLPClassifier(max_iter=1000, random_state=0)

The error message is telling you that the target data is multiclass but you are using the "binary" average option for precision. In a multiclass classification problem, there are more than two classes and so you need to use one of the other average options (None, 'micro', 'macro', 'weighted') instead of "binary".

You can update the precision_score function call to use 'macro' or 'weighted' average, like so:

In [ ]:
# Evaluate the model on the test data
y_pred = mlp.predict(X_test)
#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy: ", accuracy)
#print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))

In [ ]:
#Print Eveluation metric
Print_metric("MLP",y_test, y_pred)

Evaluation metrics of the  MLP
Accuracy:  0.8098288354510134
F1 Score:  0.8065200533576378
Precision:  0.8073912549113796
Recall:  0.8061433877641324
Confusion Matrix: 
 [[1576    2    6   21   22    2]
 [   3 1597    4    6    9    3]
 [   5    8 1311  105  119    9]
 [  22   10  104  784  571   54]
 [  16   14  101  485  931    7]
 [   2    4   12   55   30 1513]]


This code uses a bag-of-words model to convert the text data into numerical representations, which are then fed into the MLP classifier. The MLP classifier is trained on the training data and evaluated on the test data. The accuracy of the model is calculated and a confusion matrix is produced to provide a more detailed analysis of the performance of the model.

Please note that this is just a basic implementation, and there are many different ways to improve and optimize the model, including tuning the hyperparameters, using different text representation methods, and combining it with other machine learning algorithms.

##**LSTM (Long Short-Term Memory)**

* LSTM (Long Short-Term Memory) for prediction, particularly in the case of sequential data, such as time series or text data.
* LSTMs are a type of Recurrent Neural Network (RNN) that are designed to handle sequential data by incorporating memory cells that allow the network to remember previous inputs.

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(text, labels, test_size=0.2)

In [ ]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
# Pad the sequences to the same length
max_len = max([len(x) for x in X_train])
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
# Train the LSTM model
model = Sequential()
model.add(Embedding(len(tokenizer.word_index)+1, 128, input_length=max_len))
model.add(LSTM(128, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
1191/1191 [==============================] - 2134s 2s/step - loss: -133.5364 - accuracy: 0.1665 - val_loss: -242.1561 - val_accuracy: 0.1700
Epoch 2/5
1191/1191 [==============================] - 2120s 2s/step - loss: -357.0473 - accuracy: 0.1665 - val_loss: -461.4809 - val_accuracy: 0.1700
Epoch 3/5
1191/1191 [==============================] - 2124s 2s/step - loss: -580.4653 - accuracy: 0.1665 - val_loss: -681.1412 - val_accuracy: 0.1700
Epoch 4/5
1191/1191 [==============================] - 3463s 3s/step - loss: -846.0859 - accuracy: 0.2440 - val_loss: -983.4026 - val_accuracy: 0.2960
Epoch 5/5
1191/1191 [==============================] - 2129s 2s/step - loss: -1110.2584 - accuracy: 0.3001 - val_loss: -1224.9827 - val_accuracy: 0.3141


In [ ]:
# Evaluate the model on the test data
y_prob = model.predict(X_test)
y_pred = np.argmax(y_prob, axis=-1)
#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy: ", accuracy)
#print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))

298/298 [==============================] - 36s 119ms/step


In [ ]:
#Print Eveluation metric
Print_metric("LSTM",y_test, y_pred)

Evaluation metrics of the  LSTM
Accuracy:  0.171794602541216
F1 Score:  0.04886937300235983
Precision:  0.028632433756869333
Recall:  0.16666666666666666
Confusion Matrix: 
 [[1636    0    0    0    0    0]
 [1619    0    0    0    0    0]
 [1544    0    0    0    0    0]
 [1592    0    0    0    0    0]
 [1573    0    0    0    0    0]
 [1559    0    0    0    0    0]]


C:\Users\Ntiuos\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


This code uses the Keras library to build and train an LSTM model for the prediction task. The text data is tokenized and padded to the same length, and then fed into the LSTM network. The network consists of an embedding layer, an LSTM layer, and a dense layer with a sigmoid activation function. The model is trained on

##**SVM**

Support Vector Machines (SVM) is a popular supervised machine learning algorithm used for binary classification problems, including cyber bulling prediction from tweets. In an SVM model, a hyperplane is used to separate the data into two classes, positive and negative, by maximizing the margin between them.

In [ ]:
# Preprocessing the tweets
text = tweets_Data["tweet_text"].str.lower()
le = LabelEncoder()
labels = le.fit_transform(tweets_Data["cyberbullying_type"])
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(text, labels, test_size=0.2)

In [ ]:
# Convert the text into numerical features using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
# Train the SVM model
svm = SVC()
svm.fit(X_train, y_train)

# Predict the class labels for the test set
y_pred = svm.predict(X_test)

In [ ]:
# Evaluate the accuracy of the model
Print_metric("SVM",y_test, y_pred)
#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy of SVM:", accuracy)

Evaluation metrics of the  SVM
Accuracy:  0.8317757009345794
F1 Score:  0.831553174834692
Precision:  0.835013386969003
Recall:  0.8316201027813134
Confusion Matrix: 
 [[1577    2    1   15   19    0]
 [   3 1549    0    4   15    1]
 [   2    9 1326  112  123    5]
 [  54   11   45  826  598   49]
 [  14    9   81  344 1140    6]
 [   1    5    5   49   20 1503]]


Note: The code above is just an example and may not provide the best results for cyber bulling prediction from tweets. The performance of the SVM model can be improved by tuning the hyperparameters and feature engineering.

##**Convolutional Neural Networks**

Convolutional Neural Networks (CNNs) are a type of deep neural networks that are commonly used for image classification and other computer vision tasks. They can also be applied to natural language processing tasks, including cyber bullying prediction from tweets.

In [ ]:
# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(text, labels, test_size=0.2, random_state=42)

In [ ]:
# Tokenize the texts
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
# Pad the sequences to the same length
x_train = pad_sequences(sequences_train, maxlen=50)
x_test = pad_sequences(sequences_test, maxlen=50)

In [ ]:
# One-hot encoding the labels
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
y_test

array([3, 5, 1, ..., 5, 1, 3])

In [ ]:
# Define the model
inputs = Input(shape=(50,))
embedding = Embedding(1000, 128)(inputs)
conv = Conv1D(32, 3, activation="relu")(embedding)
pool = MaxPool1D(3)(conv)
global_pool = GlobalMaxPool1D()(pool)
dense = Dense(32, activation="relu")(global_pool)
outputs = Dense(1, activation="sigmoid")(dense)
model = Model(inputs, outputs)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
953/953 [==============================] - 10s 10ms/step - loss: -879527.9375 - accuracy: 0.1683 - val_loss: -4539271.5000 - val_accuracy: 0.1625
Epoch 2/5
953/953 [==============================] - 7s 8ms/step - loss: -23559370.0000 - accuracy: 0.1680 - val_loss: -58188236.0000 - val_accuracy: 0.1625
Epoch 3/5
953/953 [==============================] - 8s 8ms/step - loss: -128463008.0000 - accuracy: 0.1680 - val_loss: -232762192.0000 - val_accuracy: 0.1625
Epoch 4/5
953/953 [==============================] - 8s 8ms/step - loss: -383518688.0000 - accuracy: 0.1680 - val_loss: -595498944.0000 - val_accuracy: 0.1625
Epoch 5/5
953/953 [==============================] - 8s 8ms/step - loss: -853221312.0000 - accuracy: 0.1680 - val_loss: -1213476992.0000 - val_accuracy: 0.1625


In [ ]:
# Evaluate the model on the test set
y_pred = model.predict(x_test).round().astype(int).flatten()

#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy of CNN:", accuracy)

298/298 [==============================] - 1s 2ms/step


In [ ]:
Print_metric("CNN",y_test, y_pred)

Evaluation metrics of the  CNN
Accuracy:  0.16843431691693794
F1 Score:  0.0480512866600761
Precision:  0.02807238615282299
Recall:  0.16666666666666666
Confusion Matrix: 
 [[   0 1557    0    0    0    0]
 [   0 1604    0    0    0    0]
 [   0 1544    0    0    0    0]
 [   0 1620    0    0    0    0]
 [   0 1626    0    0    0    0]
 [   0 1572    0    0    0    0]]


C:\Users\Ntiuos\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Note: The code above is just an example and may not provide the best results for cyber bullying prediction from tweets. The performance of the CNN model can be improved by tuning the hyperparameters and feature engineering.

##**XGBoot and CatBoost**

* CatBoost and XGBoost are both popular gradient boosting algorithms used for various machine learning tasks, including text classification. In the case of predicting cyberbullying from tweets, both CatBoost and XGBoost can be used to train a model on a labeled dataset of tweets and predict whether a new tweet contains cyberbullying or not.

* To use CatBoost for this task, one can first preprocess the tweets to convert them into numerical features, such as word embeddings, that can be fed into the CatBoost model. Then, the model can be trained on the labeled dataset and evaluated on a held-out test set. One can tune the hyperparameters of the model using techniques such as cross-validation to achieve the best performance.

* Similarly, XGBoost can also be used for this task. XGBoost is known for its ability to handle sparse datasets and handle categorical variables well, which can be useful in the context of text classification. As with CatBoost, one would preprocess the tweets and convert them into numerical features, then train an XGBoost model on the labeled dataset and evaluate it on a held-out test set. Hyperparameter tuning can be performed to achieve the best performance.

* Both CatBoost and XGBoost can provide high accuracy and good performance in this task, and the choice between the two would depend on the specifics of the dataset and the problem being solved.

To use CatBoost and XGBoost for predicting cyberbullying in tweets, you would need to perform the following steps:

* Preprocessing: Clean and preprocess the tweet data, including removing stopwords, punctuation, and special characters. You may also want to convert the tweets into numerical features such as word embeddings or term frequency-inverse document frequency (TF-IDF) vectors.

* Split data: Split the preprocessed data into a training set and a test set. The training set will be used to train the models, and the test set will be used to evaluate the models.

* Training: Train CatBoost and XGBoost models on the training data. You can use grid search or other hyperparameter tuning techniques to find the best hyperparameters for each model.

* Evaluation: Evaluate the performance of each model on the test data. You can use metrics such as accuracy, precision, recall, and F1 score to compare the models.

* Prediction: Once you have a well-performing model, you can use it to predict whether new tweets contain cyberbullying or not.

It's important to note that the specific implementation of each step can vary depending on the specifics of the dataset and the problem. However, these are the general steps for using CatBoost and XGBoost for text classification tasks.

**XGBoost and CatBoost to predict cyberbullying in tweets using Python:**

* In this example, the tweets are first preprocessed by converting them into TF-IDF features.
* Then, the data is split into a training set and a test set.
* The XGBoost and CatBoost models are trained on the training data, and their performance is evaluated on the test data using accuracy and F1 score metrics.

**Perform grid search with cross-validation to tune hyperparameters for both XGBoost and CatBoost:**

In [ ]:
# define the features and target
#X = tweets_Data .drop("cyberbullying_type", axis=1)
#y = tweets_Data ["cyberbullying_type"]


In [ ]:
# Preprocessing: Convert tweets to numerical features using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(tweets_Data["tweet_text"])
y = tweets_Data["cyberbullying_type"]

In [ ]:
# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# specify the hyperparameters to be tuned and their search ranges
xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [2, 3, 4],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'reg_lambda': [0.1, 1.0, 10.0],
}
cat_param_grid = {
    'depth': [2, 3, 4],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bylevel': [0.8, 1.0],
    'reg_lambda': [0.1, 1.0, 10.0],
}

In [ ]:
# initialize the XGBoost classifier
xgb_model = XGBClassifier()
cat_model = CatBoostClassifier()

In [ ]:
# perform grid search with cross-validation for XGBoost98546+
xgb_grid = GridSearchCV(xgb_model, xgb_param_grid, cv=5, scoring='accuracy',n_jobs=-1)

In [ ]:
#fit the model
xgb_grid.fit(X_train, y_train)

In [ ]:
# display the best hyperparameters and performance
print("Best hyperparameters and performance for XGBoost:")
print(xgb_grid.best_params_)
print(xgb_grid.best_score_)

In [ ]:
# perform grid search with cross-validation for CatBoost
cat_grid = GridSearchCV(cat_model, cat_param_grid, cv=5, scoring='accuracy',n_jobs=-1)

In [ ]:
cat_grid.fit(X_train, y_train)

In [ ]:
# display the best hyperparameters and performance
print("Best hyperparameters and performance for CatBoost:")
print(cat_grid.best_params_)
print(cat_grid.best_score_)

In this example, GridSearchCV is used to perform a search over the specified hyperparameter ranges with cross-validation (cv=5) to evaluate the performance of the models. The best hyperparameters and performance for both XGBoost and CatBoost will be displayed after the grid search.

**Train Best Hyperparamter**

In [ ]:
# Preprocessing: Convert tweets to numerical features using TF-IDF
#vectorizer = TfidfVectorizer()
#tweets = vectorizer.fit_transform(tweets_Data["tweet"])
#labels = tweets_Data["label"]

In [ ]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train XGBoost model
xgb = XGBClassifier(xgb_grid.best_params_)
xgb.fit(X_train, y_train)

In [ ]:
# Train CatBoost model
cb = CatBoostClassifier(cat_grid.best_params_, verbose=0)
cb.fit(X_train, y_train)

In [ ]:
# Make predictions on test data
xgb_preds = xgb.predict(X_test)
cb_preds = cb.predict(X_test)

In [ ]:
# Print evaluation metrics
Print_metric("XGBoost",y_test, xgb_preds)

In [ ]:
# Print evaluation metrics
Print_metric("CatBoost",y_test, cb_preds)

#Evaluate models
#XGBoost
xgb_acc = accuracy_score(y_test, xgb_preds)
xgb_f1 = f1_score(y_test, xgb_preds)
xgb_acc_pre = precision_score(y_test, xgb_preds)
xgb_acc_recall = recall_score(y_test, xgb_preds)

#Print results
print("XGBoost Accuracy: ", xgb_acc)
print("XGBoost F1 Score: ", xgb_f1)
print("XGBoost Precision: ", xgb_acc_pre)
print("XGBoost Recall: ", xgb_acc_recall)  
print("Confusion Matrix: \n", confusion_matrix(y_test, xgb_preds))

#Evaluate models
#Catboost
cb_acc = accuracy_score(y_test, cb_preds)
cb_f1 = f1_score(y_test, cb_preds)
cb_pre = precision_score(y_test, cb_preds)
cb_f1 = recall_score(y_test, cb_preds)

#Print results
print("CatBoost Accuracy: ", cb_acc)
print("CatBoost F1 Score: ", cb_f1)
print("CatBoost Precision: ", cb_pre)
print("CatBoost Recall: ", cb_f1)
print("Confusion Matrix: \n", confusion_matrix(y_test, cb_preds))

##**Bi-directional LSTM**

In [ ]:
# Load the data into a pandas dataframe
data = pd.read_csv("tweets.csv")

# Preprocessing the data
max_features = 20000 # number of unique words to use
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data['text'])
list_tokenized_tweets = tokenizer.texts_to_sequences(data['text'])

# Padding the sequences to have the same length
maxlen = 100 # maximum number of words in a tweet
X = pad_sequences(list_tokenized_tweets, maxlen=maxlen)

# Defining the model
inp = Input(shape=(maxlen,))
x = Embedding(max_features, 128)(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, data['label'], batch_size=32, epochs=10, validation_split=0.1)

In [ ]:
# Load the test data into a pandas dataframe
test_data = pd.read_csv("test_tweets.csv")

# Preprocess the test data in the same way as the training data
list_tokenized_test_tweets = tokenizer.texts_to_sequences(test_data['text'])
X_test = pad_sequences(list_tokenized_test_tweets, maxlen=maxlen)

# Predict the labels for the test data
y_pred = model.predict(X_test)

# Convert the predicted probabilities to binary labels
y_pred = (y_pred > 0.5).astype(int)

# Evaluate the model performance using metrics such as accuracy, precision, recall, and F1-score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

acc = accuracy_score(test_data['label'], y_pred)
prec = precision_score(test_data['label'], y_pred)
recall = recall_score(test_data['label'], y_pred)
f1 = f1_score(test_data['label'], y_pred)

print("Accuracy: ", acc)
print("Precision: ", prec)
print("Recall: ", recall)
print("F1-score: ", f1)

##**3 Layers LSTM**

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

# Load the data into a pandas dataframe
data = pd.read_csv("tweets.csv")

# Preprocessing the data
max_features = 20000 # number of unique words to use
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data['text'])
list_tokenized_tweets = tokenizer.texts_to_sequences(data['text'])

# Padding the sequences to have the same length
maxlen = 100 # maximum number of words in a tweet
X = pad_sequences(list_tokenized_tweets, maxlen=maxlen)

# Defining the model
inp = Input(shape=(maxlen,))
x = Embedding(max_features, 128)(inp)
x = LSTM(60, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(x)
x = LSTM(60, return_sequences=True, dropout=0.1, recurrent_dropout=0.1)(x)
x = LSTM(60, return_sequences=False, dropout=0.1, recurrent_dropout=0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, data['label'], batch_size=32, epochs=10, validation_split=0.1)


The number of neurons in each layer of the LSTM network is a hyperparameter that needs to be tuned based on the specific requirements of your task. In the code example provided, the number of neurons in each LSTM layer is set to 60. This was likely chosen as a reasonable starting point for this specific problem, but there's no universal rule for choosing the number of neurons in LSTM layers.

In general, having more neurons in the LSTM layers can increase the capacity of the network to model complex relationships in the data, but it can also increase the risk of overfitting. On the other hand, having fewer neurons can limit the network's ability to model complex relationships but can reduce the risk of overfitting.

The optimal number of neurons for a specific task is often determined through experimentation and tuning, for example, by trying different values and evaluating the model performance using a validation set.

 ##**Transfer learning**

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.models import Model
from keras.applications import Xception
from keras import initializers, regularizers, constraints, optimizers, layers

# Load the data into a pandas dataframe
data = pd.read_csv("tweets.csv")

# Preprocessing the data
max_features = 20000 # number of unique words to use
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data['text'])
list_tokenized_tweets = tokenizer.texts_to_sequences(data['text'])

# Padding the sequences to have the same length
maxlen = 100 # maximum number of words in a tweet
X = pad_sequences(list_tokenized_tweets, maxlen=maxlen)

# Load the pre-trained Xception model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(maxlen,))

# Add a custom head for the binary classification task
x = base_model.output
x = GlobalAveragePooling1D()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Freeze the base model and compile the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the transfer learning model
model.fit(X, data['label'], batch_size=32, epochs=10, validation_split=0.1)

The error is because the f1_score function from scikit-learn's metrics module expects binary class labels but the target variable in this case is multiclass. You can specify the average type to be 'macro' or 'weighted' like this:

In [ ]:
f1 = f1_score(y_test, y_pred, average='macro')


Or you can set average to None to get the f1-score for each class and then average them manually:

In [ ]:
f1 = f1_score(y_test, y_pred, average=None)
f1_mean = f1.mean()
